# 📄 PDF RAG Pipeline

In [ ]:
import os
import pathlib
import fitz  # PyMuPDF
import chromadb
from chromadb.utils import embedding_functions
from tqdm import tqdm
import re
import requests
from typing import List
import numpy as np
import pathlib
from dotenv import load_dotenv

from langchain.prompts import PromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.schema import Document
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

print("✅ All libraries imported successfully")



c:\Users\ASUS\anaconda3\envs\env1\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


✅ All libraries imported successfully


In [ ]:
"""
Configuration Settings
"""
# Load environment variables
load_dotenv()

# API Configuration
API_KEY = os.getenv("API_KEY")
if not API_KEY:
    raise ValueError("API_KEY not found.") 
PDF_DIR = "pdfs"
CHROMA_PATH = "./chroma_db"

# Text Processing Parameters
CHUNK_SIZE = 600
CHUNK_OVERLAP = 100
TOP_K_RESULTS = 3

# LLM Parameters
MODEL_NAME = "anthropic/claude-3.5-sonnet"
TEMPERATURE = 0.0
MAX_TOKENS = 600

# Embedding Parameters
EMBEDDING_MODEL = "openai/text-embedding-3-small"

# Create necessary directories
pathlib.Path(PDF_DIR).mkdir(parents=True, exist_ok=True)
pathlib.Path(CHROMA_PATH).mkdir(parents=True, exist_ok=True)

print(f"📁 PDF Directory: {PDF_DIR}")
print(f"💾 ChromaDB Path: {CHROMA_PATH}")

📁 PDF Directory: pdfs
💾 ChromaDB Path: ./chroma_db


In [3]:
def generate_embeddings(texts: List[str], api_key: str, model_name: str = EMBEDDING_MODEL) -> List[List[float]]:
    """
    Generate embeddings for input texts using OpenRouter API.
    
    Args:
        texts: List of text strings to embed
        api_key: OpenRouter API key
        model_name: Model identifier on OpenRouter
        
    Returns:
        List of embedding vectors
    """
    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/embeddings",
            headers={
                "Authorization": f"Bearer {api_key}",
                "Content-Type": "application/json",
                "HTTP-Referer": "http://localhost",
                "X-Title": "Resume-RAG-System"
            },
            json={
                "model": model_name,
                "input": texts
            },
            timeout=30
        )
        
        if response.status_code == 200:
            result = response.json()
            embeddings = [item['embedding'] for item in result.get('data', [])]
            
            if not embeddings:
                raise ValueError("No embeddings returned from API")
            
            return embeddings
        else:
            error_msg = f"API Error {response.status_code}: {response.text}"
            print(f"❌ {error_msg}")
            raise ValueError(error_msg)
            
    except requests.exceptions.Timeout:
        raise ValueError("Request timed out while generating embeddings")
    except Exception as e:
        raise ValueError(f"Embedding generation failed: {str(e)}")


# Create embedding function wrapper for ChromaDB
def create_embedding_function(api_key: str, model_name: str = EMBEDDING_MODEL):
    """Create a ChromaDB-compatible embedding function."""
    
    class OpenRouterEmbeddings(embedding_functions.EmbeddingFunction):
        def __call__(self, input: List[str]) -> List[List[float]]:
            return generate_embeddings(input, api_key, model_name)
    
    return OpenRouterEmbeddings()


print("✅ embedding function defined")

✅ embedding function defined


In [4]:
"""
PDF Processing Utilities
"""

def extract_text_from_pdf(pdf_path: str) -> str:
    """
    Extract text content from a PDF file.
    
    Args:
        pdf_path: Path to the PDF file
        
    Returns:
        Extracted text as a string
    """
    try:
        doc = fitz.open(pdf_path)
        text = ""
        for page in doc:
            text += page.get_text("text", sort=True) + "\n"
        doc.close()
        return text.strip()
    except Exception as e:
        print(f"❌ Error reading {pdf_path}: {e}")
        return ""


def preprocess_text(text: str) -> str:
    """
    Clean and normalize extracted text.
    
    Args:
        text: Raw text from PDF
        
    Returns:
        Cleaned text
    """
    # Remove excessive newlines
    text = re.sub(r'\n{3,}', '\n\n', text)
    
    # Remove excessive spaces
    text = re.sub(r' {2,}', ' ', text)
    
    # Remove control characters
    text = re.sub(r'[\x00-\x08\x0b-\x0c\x0e-\x1f]', '', text)
    
    return text.strip()


def chunk_text(text: str, chunk_size: int = CHUNK_SIZE, 
               chunk_overlap: int = CHUNK_OVERLAP) -> List[str]:
    """
    Split text into overlapping chunks for better retrieval.
    
    Args:
        text: Text to split
        chunk_size: Maximum chunk size
        chunk_overlap: Overlap between chunks
        
    Returns:
        List of text chunks
    """
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size, 
        chunk_overlap=chunk_overlap,
        separators=["\n\n", "\n", ". ", " ", ""]
    )
    return splitter.split_text(text)


print("✅ Text processing functions defined")


✅ Text processing functions defined


In [5]:
"""
LLM Invocation Function
"""

def invoke_llm(prompt: str, model_name: str = MODEL_NAME, 
               temperature: float = TEMPERATURE, 
               max_tokens: int = MAX_TOKENS) -> str:
    """
    Generate a response from the LLM using OpenRouter API.
    
    Args:
        prompt: Input prompt string
        model_name: Model identifier on OpenRouter
        temperature: Sampling temperature (0.0 = deterministic)
        max_tokens: Maximum tokens in response
        
    Returns:
        Generated text response
    """
    try:
        response = requests.post(
            "https://openrouter.ai/api/v1/chat/completions",
            headers={
                "Authorization": f"Bearer {OS_API_KEY}",
                "Content-Type": "application/json",
                "HTTP-Referer": "http://localhost",
                "X-Title": "Resume-RAG-System"
            },
            json={
                "model": model_name,
                "messages": [{"role": "user", "content": prompt}],
                "max_tokens": max_tokens,
                "temperature": temperature
            },
            timeout=30
        )
        
        if response.status_code == 200:
            return response.json()["choices"][0]["message"]["content"].strip()
        else:
            return f"API Error: {response.status_code} - {response.text}"
            
    except requests.exceptions.Timeout:
        return "Error: Request timed out"
    except Exception as e:
        return f"Error: {str(e)}"


print("✅ LLM function defined")


✅ LLM function defined


In [6]:
"""
ChromaDB Retrieval Function
"""

def retrieve_documents(query: str, collection, k: int = TOP_K_RESULTS) -> List[Document]:
    """
    Retrieve relevant documents for a query from ChromaDB.
    
    Args:
        query: Search query string
        collection: ChromaDB collection instance
        k: Number of documents to retrieve
        
    Returns:
        List of LangChain Document objects
    """
    results = collection.query(
        query_texts=[query],
        n_results=k
    )
    
    documents = []
    if results['documents'] and results['documents'][0]:
        for i, doc_text in enumerate(results['documents'][0]):
            metadata = results['metadatas'][0][i] if results['metadatas'] else {}
            documents.append(
                Document(page_content=doc_text, metadata=metadata)
            )
    
    return documents


print("✅ Retriever function defined")


✅ Retriever function defined


In [7]:
"""
RAG Prompt Template
"""

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""You are a professional Resume Screening Assistant specialized in analyzing candidate qualifications.

Context from Resume(s):
{context}

Question: {question}

Instructions:
- Answer using ONLY the information provided in the context above
- Be specific and cite relevant details from the resume
- If the requested information is not available in the context, explicitly state: "This information is not available in the resume"
- Do not make assumptions or add information not present in the context
- Keep your answer concise and focused on the question

Answer:"""
)

print("✅ Prompt template created")
print(f"\nTemplate variables: {prompt_template.input_variables}")

✅ Prompt template created

Template variables: ['context', 'question']


In [8]:
"""
Helper function to format retrieved documents
"""

def format_docs(docs: List[Document]) -> str:
    """
    Format retrieved documents into a context string.
    
    Args:
        docs: List of LangChain Document objects
        
    Returns:
        Formatted context string
    """
    if not docs:
        return "No relevant information found in the resumes."
    
    formatted_chunks = []
    for i, doc in enumerate(docs, 1):
        source = doc.metadata.get('doc_name', 'Unknown')
        formatted_chunks.append(
            f"[Chunk {i} from {source}]:\n{doc.page_content}"
        )
    
    return "\n\n".join(formatted_chunks)


print("✅ Document formatter defined")


✅ Document formatter defined


In [9]:
"""
Complete RAG Pipeline using LangChain LCEL
"""

def build_rag_chain(collection, prompt: PromptTemplate, k: int = TOP_K_RESULTS):
    """
    Construct a complete RAG chain using LangChain Expression Language.
    
    """
    
    # Create retriever function
    def retriever_fn(query: str) -> List[Document]:
        return retrieve_documents(query, collection, k)
    
    # Convert PromptValue to string before passing to LLM
    def prompt_to_string(prompt_value):
        """Convert LangChain PromptValue to plain string."""
        return prompt_value.to_string()
    
    # Wrap functions in RunnableLambda to make them LCEL-compatible
    retriever_runnable = RunnableLambda(retriever_fn)
    format_docs_runnable = RunnableLambda(format_docs)
    prompt_string_runnable = RunnableLambda(prompt_to_string)
    llm_runnable = RunnableLambda(invoke_llm)
    
    # Build the chain using LCEL syntax
    rag_chain = (
        {
            "context": retriever_runnable | format_docs_runnable,
            "question": RunnablePassthrough()
        }
        | prompt
        | prompt_string_runnable
        | llm_runnable
        | StrOutputParser()
    )
    
    return rag_chain


print("✅ RAG chain builder defined")


✅ RAG chain builder defined


In [10]:
"""
Setup ChromaDB with Custom OpenRouter Embeddings
"""

# Initialize custom embedding function
embedding_function = create_embedding_function(
    api_key=OS_API_KEY,
    model_name=EMBEDDING_MODEL
)

# Initialize ChromaDB client
client = chromadb.PersistentClient(path=CHROMA_PATH)

print("✅ Custom embedding function and ChromaDB client initialized")

✅ Custom embedding function and ChromaDB client initialized


C:\Users\ASUS\AppData\Local\Temp\ipykernel_5664\2060399078.py:56: DeprecationWarning: The class OpenRouterEmbeddings does not implement __init__. This will be required in a future version.
  return OpenRouterEmbeddings()


In [11]:
"""
Load and Index Resume PDFs
"""

# Check for PDF files
pdf_files = [f for f in os.listdir(PDF_DIR) if f.lower().endswith(".pdf")]

if not pdf_files:
    print(f"❌ No PDFs found in {PDF_DIR}!")
    print(f"Please add resume PDFs to the '{PDF_DIR}' directory.")
else:
    # Reset collection
    try:
        client.delete_collection("resume_index")
        print("🗑️  Deleted existing collection")
    except:
        pass
    
    # Create new collection with custom embedding function
    collection = client.create_collection(
        name="resume_index", 
        embedding_function=embedding_function
    )
    
    # Index PDFs with batch processing
    print(f"\n📄 Indexing {len(pdf_files)} PDF(s)...")
    
    total_chunks = 0
    batch_size = 10  # Process chunks in batches to avoid API limits
    
    for pdf_name in tqdm(pdf_files, desc="Processing PDFs"):
        pdf_path = os.path.join(PDF_DIR, pdf_name)
        
        # Extract and process text
        raw_text = extract_text_from_pdf(pdf_path)
        if not raw_text:
            print(f"⚠️  Skipping {pdf_name} - no text extracted")
            continue
            
        cleaned_text = preprocess_text(raw_text)
        chunks = chunk_text(cleaned_text)
        
        if chunks:
            # Add chunks in batches
            for i in range(0, len(chunks), batch_size):
                batch_chunks = chunks[i:i + batch_size]
                batch_ids = [f"{pdf_name}_chunk_{j}" for j in range(i, i + len(batch_chunks))]
                batch_metadata = [{"doc_name": pdf_name} for _ in batch_chunks]
                
                try:
                    collection.add(
                        documents=batch_chunks,
                        metadatas=batch_metadata,
                        ids=batch_ids
                    )
                    total_chunks += len(batch_chunks)
                except Exception as e:
                    print(f"\n❌ Error adding batch from {pdf_name}: {e}")
                    continue
    
    print(f"\n✅ Successfully indexed {total_chunks} chunks from {len(pdf_files)} PDF(s)")
    print(f"📊 Collection size: {collection.count()} documents")


🗑️  Deleted existing collection

📄 Indexing 12 PDF(s)...


Processing PDFs: 100%|██████████| 12/12 [00:14<00:00,  1.18s/it]


✅ Successfully indexed 80 chunks from 12 PDF(s)
📊 Collection size: 80 documents


In [12]:
"""
Assemble All Components into RAG Chain
"""

# Build the chain
rag_chain = build_rag_chain(collection, prompt_template, k=TOP_K_RESULTS)

print("✅ RAG chain assembled successfully!")
print("\n🔗 Chain Components:")
print("  1. Retriever: ChromaDB vector search")
print("  2. Formatter: Document context builder")
print("  3. Prompt: Resume screening template")
print("  4. LLM: Claude 3.5 Sonnet via OpenRouter")
print("  5. Parser: String output parser")


✅ RAG chain assembled successfully!

🔗 Chain Components:
  1. Retriever: ChromaDB vector search
  2. Formatter: Document context builder
  3. Prompt: Resume screening template
  4. LLM: Claude 3.5 Sonnet via OpenRouter
  5. Parser: String output parser


In [13]:
"""
Interactive Resume Query System
"""

def run_interactive_qa():
    """
    Run an interactive question-answering session.
    """
    print("\n" + "=" * 80)
    print("INTERACTIVE RESUME SCREENING SYSTEM")
    print("=" * 80)
    print("\nAsk questions about the resumes (type 'exit', 'quit', or 'q' to stop)")
    print("=" * 80 + "\n")
    
    while True:
        question = input("❓ Your question: ").strip()
        
        if not question:
            continue
        
        if question.lower() in ['exit', 'quit', 'q']:
            print("\n👋 Goodbye!")
            break
        
        try:
            print("\n🔄 Processing...\n")
            answer = rag_chain.invoke(question)
            print(f"💡 Answer:\n{answer}\n")
            print("-" * 80 + "\n")
            
        except Exception as e:
            print(f"❌ Error: {e}\n")

In [14]:
print("✅ Interactive Q&A function ready")
run_interactive_qa()

✅ Interactive Q&A function ready

INTERACTIVE RESUME SCREENING SYSTEM

Ask questions about the resumes (type 'exit', 'quit', or 'q' to stop)


🔄 Processing...

💡 Answer:
According to the resume context, Bissan's email is: salehbissan22@gmail.com

--------------------------------------------------------------------------------


👋 Goodbye!
